<div style="width:90%;
            margin:10px;
            padding:8px;
            border:2px solid #FF0000;
            border-radius:20px;
            font-weight:bold;
            font-size:2.5em;
            text-align:center;">
The Quadrotor Problem
</div>


<div style="width:90%;
            margin:9px;
            font-size:1em;">
            
*The Quadrotor Problem:* The goal is to move the quadrotor from a given initial position and orientation to a given final position and orientation in minimum total time while avoiding cylindrical obstacles.                              
The problem is inspired from the following paper: [FATROP : A Fast Constrained Optimal Control Problem Solver for Robot Trajectory Optimization and Control](https://arxiv.org/pdf/2303.16746) made for [Casadi](https://web.casadi.org/) software and [Fatrop](https://github.com/meco-group/fatrop) solver.                            
We solve the problem using the [JuMP](https://jump.dev/) and [OptimalControl](https://control-toolbox.org/OptimalControl.jl/stable/) packages in Julia then we compare the results.

1. [JuMP Formulation](#jump)
2. [OptimalControl Formulation](#oc)
3. [Comparison](#comp)

</div>

## 1. Solving with JuMP <a id='jump'></a>

In [1]:
using JuMP , Ipopt
include("../Problems/JuMP/quadrotor_JMP.jl")
QuadrotorJMP = quadrotor_JMP();


In [2]:
import HSL_jll
set_optimizer(QuadrotorJMP,Ipopt.Optimizer)
set_attribute(QuadrotorJMP, "print_level", 5)
set_optimizer_attribute(QuadrotorJMP,"tol",1e-8)
set_optimizer_attribute(QuadrotorJMP,"constr_viol_tol",1e-6)
set_optimizer_attribute(QuadrotorJMP,"max_iter",1000)
set_optimizer_attribute(QuadrotorJMP,"mu_strategy","adaptive")
set_attribute(QuadrotorJMP, "hsllib", HSL_jll.libhsl_path)
set_attribute(QuadrotorJMP, "linear_solver", "ma57")

In [3]:
println("Solving...")
optimize!(QuadrotorJMP)
println()

Solving...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:    59367
Number of nonzeros in inequality constraint Jacobian.:      802
Number of nonzeros in Lagrangian Hessian.............:  7233157

Total number of variables............................:     1011
                     variables with only lower bounds:        1
                variables with lower and upper bounds:      303
                     variables with only upper bounds:        0
Total number of equality constraints.................:      617
Total number of inequali

In [ ]:
# Display results
if termination_status(QuadrotorJMP) == MOI.OPTIMAL
    println("  Solution is optimal")
elseif  termination_status(QuadrotorJMP) == MOI.LOCALLY_SOLVED
    println("  (Local) solution found")
elseif termination_status(QuadrotorJMP) == MOI.TIME_LIMIT && has_values(QuadrotorJMP)
    println("  Solution is suboptimal due to a time limit, but a primal solution is available")
else
    error("  The model was not solved correctly.")
end
println("  objective value = ", objective_value(QuadrotorJMP))
println()

## 2. Solving with OptimalControl <a id='oc'></a>

In [23]:
using OptimalControl
include("../Problems/OptimalControl/quadrotor_OC.jl")
QuadrotorOC = quadrotor_OC();

In [24]:
import HSL_jll
# Solve the problem
sol = OptimalControl.solve(QuadrotorOC, max_iter = 2,
            linear_solver="ma57",hsllib=HSL_jll.libhsl_path)

Method = (:direct, :adnlp, :ipopt)
This is Ipopt version 3.14.14, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:     5110
Number of nonzeros in inequality constraint Jacobian.:      202
Number of nonzeros in Lagrangian Hessian.............:     1314

Total number of variables............................:     1112
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      303
                     variables with only upper bounds:        0
Total number of equality constraints.................:      713
Total number of inequality constraints...............:      101
        inequality constraints with only lower bounds:      101
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.0000000e-01 4.90e+00 1.00e+00   0.0 0.00e

CTBase.OptimalControlSolution

In [ ]:
plot(sol)